# Customer Churn Prediction

## Random Forest Classifier implimentation

#### Loading Data

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
import pickle as pickle 
import sys
import os
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from math import sin, cos, sqrt, atan2, radians
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn import metrics

Using TensorFlow backend.


In [4]:
data_= pd.read_csv('data/customer_preprocessed_data.csv')


In [6]:
data = data_[['passengerid', 'trip_total', 'delay_total', 'delay_avg',
       'distance_total', 'distance_avg', 'fare_total', 'fare_avg',
       'discount_total', 'discount_avg', 'preffered_taximodelid',
       'trip_fare_total', 'trip_fare_avg',
       'preffered_payment_method', 'profile_engaged_period', 'pre_booked',
       'churn']]

In [7]:
data.corr()

,passengerid,trip_total,delay_total,delay_avg,distance_total,distance_avg,fare_total,fare_avg,discount_total,discount_avg,preffered_taximodelid,trip_fare_total,trip_fare_avg,preffered_payment_method,profile_engaged_period
passengerid,1.000000,0.000152,-0.000071,-0.000285,-0.000163,-0.000648,0.002237,-0.001109,-0.001108,-0.001114,0.000184,0.002532,-0.001026,0.001388,-0.003446
trip_total,0.000152,1.000000,0.659390,-0.014032,0.674052,-0.024680,0.011140,0.001301,0.003418,0.002085,-0.072747,0.010655,0.001141,0.106957,0.107615
delay_total,-0.000071,0.659390,1.000000,0.326255,0.500756,-0.006194,0.008769,0.001119,0.001981,0.000914,-0.021032,0.008543,0.001050,0.071620,0.071952
delay_avg,-0.000285,-0.014032,0.326255,1.000000,0.005507,0.018111,0.000007,-0.000039,-0.000183,-0.000128,0.061060,0.000047,-0.000029,-0.003476,-0.005572
distance_total,-0.000163,0.674052,0.500756,0.005507,1.000000,0.505399,0.011706,0.003179,0.009950,0.008334,0.010037,0.009805,0.002535,0.075225,0.076152
distance_avg,-0.000648,-0.024680,-0.006194,0.018111,0.505399,1.000000,0.000893,0.002712,0.001245,0.001747,0.079570,0.000641,0.002583,-0.001005,-0.007688
fare_total,0.002237,0.011140,0.008769,0.000007,0.011706,0.000893,1.000000,0.720850,0.213926,0.213915,0.001229,0.976848,0.706294,0.002516,0.000978
fare_avg,-0.001109,0.001301,0.001119,-0.000039,0.003179,0.002712,0.720850,1.000000,0.078123,0.078122,-0.000360,0.720824,0.996943,0.000573,0.001425
discount_total,-0.001108,0.003418,0.001981,-0.000183,0.009950,0.001245,0.213926,0.078123,1.000000,0.999976,0.000629,-0.000011,-0.000011,-0.000301,-0.001018
discount_avg,-0.001114,0.002085,0.000914,-0.000128,0.008334,0.001747,0.213915,0.078122,0.999976,1.000000,0.001062,-0.000017,-0.000014,-0.000402,-0.001088


## Label Encoding Method

In [79]:
label_encoder_no_of_class = LabelEncoder()
label_encoder_no_of_class  =label_encoder_no_of_class.fit(prediction_data['churn'])
prediction_data["churn"] = label_encoder_no_of_class.transform(prediction_data["churn"])



In [84]:
label_encoder_preffered_taximodelid = LabelEncoder()
label_encoder_preffered_taximodelid  =label_encoder_preffered_taximodelid.fit(prediction_data['preffered_taximodelid'])
prediction_data["preffered_taximodelid"] = label_encoder_preffered_taximodelid.transform(prediction_data["preffered_taximodelid"])

In [102]:
label_encoder_pre_booked = LabelEncoder()
label_encoder_pre_booked =label_encoder_pre_booked.fit(prediction_data['pre_booked'])
prediction_data["pre_booked"] = label_encoder_pre_booked.transform(prediction_data["pre_booked"])

## One-hot Encoding Method

In [8]:
data = pd.get_dummies(data=data, columns=['preffered_taximodelid','preffered_payment_method','pre_booked'])

In [9]:
data.columns

Index(['passengerid', 'trip_total', 'delay_total', 'delay_avg',
       'distance_total', 'distance_avg', 'fare_total', 'fare_avg',
       'discount_total', 'discount_avg', 'trip_fare_total', 'trip_fare_avg',
       'profile_engaged_period', 'churn', 'preffered_taximodelid_1',
       'preffered_taximodelid_2', 'preffered_taximodelid_3',
       'preffered_taximodelid_4', 'preffered_taximodelid_10',
       'preffered_taximodelid_24', 'preffered_payment_method_1.0',
       'preffered_payment_method_2.0', 'preffered_payment_method_3.0',
       'preffered_payment_method_5.0', 'preffered_payment_method_7.0',
       'pre_booked_0', 'pre_booked_0,1', 'pre_booked_1'],
      dtype='object')

In [10]:
data.head()

,passengerid,trip_total,delay_total,delay_avg,distance_total,distance_avg,fare_total,fare_avg,discount_total,discount_avg,...,preffered_taximodelid_10,preffered_taximodelid_24,preffered_payment_method_1.0,preffered_payment_method_2.0,preffered_payment_method_3.0,preffered_payment_method_5.0,preffered_payment_method_7.0,pre_booked_0,"pre_booked_0,1",pre_booked_1
0,2,3,14.72,4.906667,16.41,5.470000,661.35,220.450000,0.0,0.0,...,0,0,1,0,0,0,0,1,0,0
1,4,2,17.63,8.815000,12.26,6.130000,506.36,253.180000,200.0,100.0,...,0,0,1,0,0,0,0,1,0,0
2,5,1,6.45,6.450000,12.44,12.440000,471.85,471.850000,0.0,0.0,...,0,0,1,0,0,0,0,1,0,0
3,7,52,486.82,9.361923,280.20,5.388462,12725.76,244.726154,0.0,0.0,...,0,0,1,0,0,0,0,1,0,0
4,9,2,8.45,4.225000,6.80,3.400000,293.84,146.920000,0.0,0.0,...,0,0,1,0,0,0,0,1,0,0


In [12]:
data= data[['passengerid', 'trip_total', 'delay_total', 'delay_avg',
       'distance_total', 'distance_avg', 'fare_total', 'fare_avg',
       'discount_total', 'discount_avg', 'trip_fare_total', 'trip_fare_avg',
       'profile_engaged_period', 'preffered_taximodelid_1',
       'preffered_taximodelid_2', 'preffered_taximodelid_3',
       'preffered_taximodelid_4', 'preffered_taximodelid_10',
       'preffered_taximodelid_24', 'preffered_payment_method_1.0',
       'preffered_payment_method_2.0', 'preffered_payment_method_3.0',
       'preffered_payment_method_5.0', 'preffered_payment_method_7.0',
       'pre_booked_0', 'pre_booked_0,1', 'pre_booked_1', 'churn']]


In [14]:
label_encoder_class = LabelEncoder()
label_encoder_class  =label_encoder_class.fit(data["churn"])
data["churn"] = label_encoder_class.transform(data["churn"])

In [15]:
data['churn'].head()

0    0
1    0
2    0
3    0
4    1
Name: churn, dtype: int64

In [16]:
data.shape

(384591, 28)

## Split Data

In [45]:

X = data.iloc[:, 1:27].values 
Y = data.iloc[:, 27].values

#Splitting the dataset into the Training set and Test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3490, random_state = 0)

### Data Scaling

In [46]:

minmax_scale = preprocessing.MinMaxScaler().fit(X_train)
X_train = minmax_scale.transform(X_train)

### Using Default Random Forest Classification

In [37]:
clf = RandomForestClassifier(n_jobs=2, random_state=0)

In [38]:
clf = clf.fit(X_train,Y_train)

In [39]:
X_test = minmax_scale.transform(X_test)

In [40]:
Y_pred = clf .predict(X_test)

In [41]:
#confusion_matrix_out = confusion_matrix(Y_test, Y_pred)

In [42]:
Accuracy= metrics.accuracy_score(Y_test, Y_pred)
Precision= metrics.precision_score(Y_test, Y_pred)
Recall= metrics.recall_score(Y_test, Y_pred)
f1score = metrics.f1_score(Y_test, Y_pred, average='macro')

In [43]:
print('Accuracy is:', Accuracy)
print('Precision is: ', Precision)
print('Recall is: ', Recall)
print('F1score is: ', f1score)

Accuracy is: 0.7202267867653085
Precision is:  0.6482749015082176
Recall is:  0.621111026384017
F1score is:  0.7039098226814149


In [44]:
pickle.dump(clf, open("model/random_forest_classifier.pkl","wb"))

### Using Hyper parameter optimization

In [47]:
clf = RandomForestClassifier()

In [49]:
param_dist = {"clf__max_features": sp_randint(1, 20),
              "clf__bootstrap": [True, False],
              "clf__warm_start":[True, False],
              "clf__n_estimators":sp_randint(50, 150),
              "clf__min_samples_split" : sp_randint(2,20),
              "clf__criterion": ["gini","entropy"],
              "clf__max_depth":sp_randint(1, 50),
              "clf__max_samples":sp_randint(2,100)
             }

In [50]:
pipeline = Pipeline([
        ('clf', clf)
    ])
  
# run randomized search
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist,n_iter=10, error_score=0.0, random_state=0)

In [51]:
random_search.fit(X_train,Y_train)

RandomizedSearchCV(cv=None, error_score=0.0,
                   estimator=Pipeline(memory=None,
                                      steps=[('clf',
                                              RandomForestClassifier(bootstrap=True,
                                                                     ccp_alpha=0.0,
                                                                     class_weight=None,
                                                                     criterion='gini',
                                                                     max_depth=None,
                                                                     max_features='auto',
                                                                     max_leaf_nodes=None,
                                                                     max_samples=None,
                                                                     min_impurity_decrease=0.0,
                                                           

In [52]:
print(random_search.best_score_)

0.7281122028188648


In [53]:
X_test = minmax_scale.transform(X_test)
Y_pred = random_search.best_estimator_.predict(X_test)

In [56]:
Accuracy= metrics.accuracy_score(Y_test, Y_pred)
Precision= metrics.precision_score(Y_test, Y_pred)
Recall= metrics.recall_score(Y_test, Y_pred)
f1score = metrics.f1_score(Y_test, Y_pred, average='macro')

In [57]:
print('Accuracy is:', Accuracy)
print('Precision is: ', Precision)
print('Recall is: ', Recall)
print('F1score is: ', f1score)

Accuracy is: 0.7266787361331516
Precision is:  0.654359657022043
Recall is:  0.6372197651364954
F1score is:  0.71160627476979


In [55]:
random_search.best_estimator_

Pipeline(memory=None,
         steps=[('clf',
                 RandomForestClassifier(bootstrap=False, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=10, max_features=17,
                                        max_leaf_nodes=None, max_samples=71,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1,
                                        min_samples_split=17,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=97, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=True))],
         verbose=False)

In [58]:

###random_search= pickle.load(open("classifier.pkl","rb"))
pickle.dump(random_search, open("model/RF_hyper_classifier.pkl","wb"))